<h1> HHC Data Investigation Process </h1>

This process is to analyze and investigate the cleaned data that has been loaded into a local postgres rds.

<h4> #1 Import necessary packages </h4>

* getpass is used to store and pass DB password
* numpy is a aggregation framework that works with pandas
* pandas a dataframe package used in this process for json to rds mapping
* psycopg2 package used to interact with Postgres DBS, the rds used in this project
* plotly package is used for data visualization and graphing

    
<h4> #2 Setup external connections </h4>

* Establish connection to our postgres database: hhc_db1
* Establish connection to plotly API

<h4> #3 Initial visit investigation </h4>

* build necessary functions
* connect to hhc_db1 and pull data from mexia_visit table
* clean the data and re-orient into dataframe
* tabulate high-level metrics
  * Total Time Spent
  * Total Number of Visitors
  * Total Unique Visitors
  * Average Visits per person

    
<h4> #4 Zone Deep Dive </h4>

* build necessary functions
* apply zone function to current_zone_group
* derive Zone Type time percentages
* build line chart and display

<h4> #5 Location Study (Merged Data Analysis)</h4>

* pull relevant data from mexia and ssi tables for all Merged Data Analysis
* process subset of data for location study and remove local individuals
* build bar chart and display

<h4> #6 Gender Effect (Merged Data Analysis) </h4>

* extract subset of data for gender study and remove missing genders
* process group by unique counts and split data sets
* build grouped bar chart and display

<h4> #7 Close connection to Postgres RDS </h4>

<h2> Step 1 </h2>

In [1]:
from getpass import getpass
import numpy as np
import pandas as pd
import psycopg2
import plotly.plotly as py
import plotly.graph_objs as go

<h2> Step 2 </h2>

In [2]:
#connect to postgres
paPostgress = getpass('please enter the db admin password:')
conn = psycopg2.connect("dbname=hhc_db1 user=postgres password=%s" %paPostgress)
cur = conn.cursor()

please enter the db admin password:········


In [3]:
#connect to plotly API
plotlyPass = getpass('please enter the ploty API secret key:')
py.sign_in('CJ_Foley','%s' %plotlyPass)

please enter the ploty API secret key:········


<h2> Step 3 - Summary Analysis </h2>

<h6> Build Functions </h6>

In [4]:
#These function is designed to further clean and refine the data.

def removeCoordinates(x):
    '''
    Head:This function was built to remove the coordinate 
         on the end of the zone name
    
    Note:This function should be used with a 
         .map() or .apply() function on a pandas dataframe
    
    Requires:
        single data point - zone_data
        
    Returns:
        single data point - stripped zone_data
    
    '''
    piecedString = x.split()
    if piecedString[-1] in ('N','S','E','W'):
        x = x[0:-1].strip()
    elif piecedString[-1] in ('NE','NW','SE','SW'):
        x = x[0:-2].strip()
    else:
        pass
    return x.strip()

<h6> Extract Relevant Data </h6>

In [5]:
#determine columns
relevantColumns = ['current_zone_name',
                   'dwell_time_in_sec',
                   'data_file_date',
                   'mac_address']

#reformat columns into string
columnsStr = ', '.join(['%s']*len(relevantColumns)) %tuple(relevantColumns)

#extract actual data from rds
cur.execute('''select %s from mexia_ssi_data;''' %columnsStr)

<h6> Clean Data </h6>

In [6]:
#load extract into dataFrame
rawExtract = pd.DataFrame(cur.fetchall(),columns=relevantColumns)

#clean extract
rawExtract['current_zone_grouped']= rawExtract['current_zone_name'].apply(removeCoordinates)

<h6> Process and Analyze Data </h6>

In [7]:
#define the aggregations we want to perform
funcs = {'dwell_time_in_sec':['sum','mean','count'],
         'mac_address':['nunique'],
         'data_file_date':['min','max']}

#performat aggregation
processedData = rawExtract.groupby(['current_zone_grouped']).agg(funcs)

#refine data
cleanData = pd.DataFrame(processedData.reset_index().values,
                         columns=['zone_name',
                                  'time_spent',
                                  'avg_time_spent',
                                  'visits',
                                  'min_visit_date',
                                  'max_visit_date',
                                  'unique_visitors'])

#perform additional calculation
cleanData['avgVisitsPerVisitor'] = cleanData['visits']/cleanData['unique_visitors']

<h6> Display Results </h6>

In [8]:
cleanData

,zone_name,time_spent,avg_time_spent,visits,min_visit_date,max_visit_date,unique_visitors,avgVisitsPerVisitor
0,American Eagle,8410620,271.564,30971,2016-12-01 00:00:00,2017-01-11 00:00:00,13388,2.31334
1,Art of Shaving,41490910,536.703,77307,2016-12-01 00:00:00,2017-01-11 00:00:00,28934,2.67184
2,B. Young,13266030,339.38,39089,2016-12-01 00:00:00,2017-01-11 00:00:00,18931,2.06481
3,Chicos,23066130,312.621,73783,2016-12-01 00:00:00,2017-01-11 00:00:00,29217,2.52534
4,Cream,53903660,574.409,93842,2016-12-01 00:00:00,2017-01-11 00:00:00,35554,2.63942
5,Dave and Buster's,52722150,403.071,130801,2016-12-01 00:00:00,2017-01-11 00:00:00,39109,3.34452
6,Express,21148830,282.414,74886,2016-12-01 00:00:00,2017-01-11 00:00:00,28394,2.63739
7,Forever 21,22785990,458.489,49698,2016-12-01 00:00:00,2017-01-11 00:00:00,17657,2.81463
8,H & M,25125620,454.787,55247,2016-12-01 00:00:00,2017-01-11 00:00:00,20799,2.65623
9,House of Hoops,30414890,325.31,93495,2016-12-01 00:00:00,2017-01-11 00:00:00,33681,2.7759


<h2> Step 4 - Zone Deep Dive </h2>

<h6> Build Functions </h6>

In [9]:
#This function is designed to bucket the zones into their zone type

def zone_type(x):
    '''
    Head:This function was built to bucket the zones 
         into their zone type after the zones have been cleaned of coordinates
    
    Note:This function should be used with a 
         .map() or .apply() function on a pandas dataframe
    
    Requires:
        single data point - stripped zone_data
        
    Returns:
        single data point - grouped zone_data
    
    '''
    if x in ['American Eagle',
          'Art of Shaving',
          'B. Young',
          'Chicos',
          'Express',
          'Forever 21',
          'H & M',
          'House of Hoops',
          'Kay Jewelers',
          'Lane Bryant',
          'Lucky Brand',
          'Lush',
          'Teavana',
          'The Buckle',
          'Toy Box',
          "Victoria's Secret"]:
        return 'Retail'

    elif x in ["Dave and Buster's",
               "The Rink",
               "Regal Theater",
               "Vent Blotique"]:
        return 'Activity'


    elif x in ['Red Rock Chocolate Factory',
               'Ribs and Burgers',
               'Cream']:
        return 'Restaurant'

    else:
        return 'Error'

<h6> Clean Data </h6>

In [10]:
deepDiveRaw = rawExtract.copy()
deepDiveRaw['zone_type']=deepDiveRaw['current_zone_grouped'].apply(zone_type)

<h6> Process and Analyze Data </h6>

In [11]:
columns = ['zone_type','data_file_date','dwell_time_in_sec']

pivotedDeepDive = pd.pivot_table(deepDiveRaw[columns],
                      index=['data_file_date'],
                      columns=['zone_type'],
                      aggfunc=np.sum
                     ).reset_index()

activeColumns = ['date',
                 'TimeSpent_Activity',
                 'TimeSpent_Restaurant',
                 'TimeSpent_Retail']

strctDpDv = pd.DataFrame(pivotedDeepDive.values,columns=activeColumns)
strctDpDv['Date']=strctDpDv['date'].apply(pd.to_datetime)

strctDpDv['RetailPercent']=100 * (strctDpDv['TimeSpent_Retail']/
                                     (strctDpDv['TimeSpent_Activity']+
                                      strctDpDv['TimeSpent_Retail'] +
                                      strctDpDv['TimeSpent_Restaurant']) )

strctDpDv['ActivityPercent']=100 * (strctDpDv['TimeSpent_Activity']/
                                          (strctDpDv['TimeSpent_Activity']+
                                           strctDpDv['TimeSpent_Retail'] +
                                           strctDpDv['TimeSpent_Restaurant']) )


strctDpDv['RestaurantPercent']=100 * (strctDpDv['TimeSpent_Restaurant']/
                                            (strctDpDv['TimeSpent_Activity']+
                                             strctDpDv['TimeSpent_Retail'] +
                                             strctDpDv['TimeSpent_Restaurant']) )
organizedData = strctDpDv[['Date',
                           'RetailPercent',
                           'ActivityPercent',
                           'RestaurantPercent']]


<H6> Display Results </h6>

In [12]:
trace0 = go.Scatter(
    x = organizedData['Date'],
    y = organizedData['RetailPercent'],
    mode = 'lines',
    name = '% Retail Visits')

trace1 = go.Scatter(
    x = organizedData['Date'],
    y = organizedData['ActivityPercent'],
    mode = 'lines',
    name = '% Activity Visits')

trace2 = go.Scatter(
    x = organizedData['Date'],
    y = organizedData['RestaurantPercent'],
    mode = 'lines',
    name = '% Restaurant Visits')

chartData = [trace0,trace1,trace2]

chartLayout = go.Layout(
    title='''Breakdown of Zone Groups Visited <br> by Date''')

zoneFig = go.Figure(data=chartData, layout=chartLayout)

py.iplot(zoneFig, filename='zone_distribution')

<h2> Step 5 - Location Study (Data Merge Analysis) </h2>

<h6> Extract Relevant Data </h6>

In [13]:
#only individuals using the SSI process will be cataloged for this analysis.

cur.execute('''
select
fbData.*,
vstData.current_zone_name,
vstData.dwell_time_in_sec,
vstData.data_file_date

from mexia_ssi_data as vstData

inner join
    (    select 
            ss_p.ssi_id,
            ss_p.fb_gender,
            ss_p.fb_loc,
            ss_m.mac_address
         from ss_profiles ss_p

         inner join (select 
                            distinct
                            ssi_id, 
                            substr(user_mac,1,2)||substr(user_mac,4,2)||substr(user_mac,7,2)||substr(user_mac,10,2)||substr(user_mac,13,2)||substr(user_mac,16,2) as mac_address   
                        from ss_mac_ref) ss_m
         on ss_p.ssi_id = ss_m.ssi_id) as fbDATA
on vstData.mac_address = fbData.mac_address

where fbData.ssi_id !=''
''')

<h6> Clean Data </h6>

In [14]:
relevantColumns = [i[0] for i in cur.description]
rawExtract = pd.DataFrame(cur.fetchall(),columns=relevantColumns)

<h6> Process and Analyze Data </h6>

In [15]:
#select necessary columns
dataSubset = rawExtract[['fb_loc','dwell_time_in_sec']]

#aggregate time spent by location
dataAgg = dataSubset.groupby('fb_loc').sum().reset_index()

#remove Las Vegas and Null
rmvdLocations = ['','Las Vegas, Nevada', 'North Las Vegas, Nevada']
dataFinal = dataAgg[~dataAgg.fb_loc.isin(rmvdLocations)]

<h6> Display Results </h6>

In [16]:
#build visual
chartData = [go.Bar(x = dataFinal['fb_loc'].values,
                    y = dataFinal['dwell_time_in_sec'].values)]
                    
chartLayout = go.Layout(
    title='''Total Time Spent by Facebook Location <br> for Non-Locals ''',
)
                    
chartFig = go.Figure(data=chartData, layout=chartLayout)

py.iplot(chartFig, filename='location_distribution')                    

<h2>Step 6 - Gender Effect (Merge Data Analysis) </h2>

<h6> Extract Relevant Data </h6>

In [17]:
dataSubsetGE = rawExtract[['fb_gender','mac_address','current_zone_name']].copy()

<h6>Clean Data </h6>

In [18]:
#remove coordinates 
dataSubsetGE['current_zone_grouped']= dataSubsetGE['current_zone_name'].apply(removeCoordinates)

#filter Out
stores = ["Victoria's Secret","Art of Shaving","Lush","House of Hoops"]
notGender = ['','N/A']
dataSubsetGeF1 = dataSubsetGE[~dataSubsetGE.fb_gender.isin(notGender)]
dataSubsetGeF2 = dataSubsetGeF1[dataSubsetGeF1.current_zone_grouped.isin(stores)]

<h6> Process and Analyze Data </h6>

In [19]:
#define the aggregations we want to perform
funcs = {'mac_address':['nunique']}

#perform aggregation
processedGe = dataSubsetGeF2.groupby(['fb_gender',
                                      'current_zone_grouped']).agg(funcs)

#refine data

cleanDataGe = pd.DataFrame(processedGe.reset_index().values,
                           columns=['Gender',
                                    'Store',
                                    'unique_visitors'])

#split data for visuals
maleData = cleanDataGe.loc[cleanDataGe['Gender']=='male']
femaleData = cleanDataGe.loc[cleanDataGe['Gender']=='female']

<h6> Display Results </h6>

In [20]:
trace0 = go.Bar(
        x= maleData['Store'].values,
        y= maleData['unique_visitors'].values,
        name='Male')

trace1 =  go.Bar(
        x= femaleData['Store'].values,
        y= femaleData['unique_visitors'].values,
        name='Female')

chartData = [trace0,trace1]

chartLayout = go.Layout(
            barmode='group',
            title='Gender Biased Stores Breakdown')


chartFig = go.Figure(data=chartData, layout=chartLayout)

py.iplot(chartFig, filename='gender_effect')  



<h2> Step 7 Close Connection </h2>

In [21]:
cur.close()
conn.close()